In [ ]:
from config import SimpleConvConfig
from models.simpleconv import SimpleConv
import torch

config = SimpleConvConfig(
    # Str to list of possible conditions
    conditions={
        "studies": ["gwilliams2023", "schoffelen2022"],
    },
    # Channels
    in_channels=208,
    out_channels=128,
    hidden_dim=256,
    dropout=0.3,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=False,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=True,
    merger_emb_dim=2048,
    merger_channels=256,
    merger_dropout=0.3,
    merger_conditional=None,
    # Inital
    initial_linear=256,
    initial_depth=1,
    # Conditional layers
    conditional_layers=True,
    conditional_layers_dim="hidden_dim",  # or hidden_dim
    # Conv layer overall structure
    depth=4,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    # Transformers Encoders
    transformer_encoder_emb="groupconv",
    transformer_encoder_layers=1,
    transformer_encoder_heads=8,
    # Transformer Decoders
    transformer_decoder_emb="groupconv",
    transformer_decoder_layers=1,
    transformer_decoder_heads=8,
    transformer_decoder_dim=256,
)

model = SimpleConv(config).to('mps')


TransEncoder 	Params: 3290112
		Spec: 256, 		Emb: groupconv, 	Bins: 16
		Layers: 1, 		D_model: 512, 		Nhead: 8

TransDecoder 	Params: 1087104
		Emb: groupconv, 	Layers: 1, 		D_model: 256
		Nhead: 8, 		Mel_bins: 128

SimpleConv: 	Params: 7725056, 	cond: ['studies']
		Merger: True, 		Merger chan: 256
		Conv blocks: 4, 	Channels: 256, 		Params: 2363904
